# Brute force pulse sequence search

Using arbitrary units. Energy is normalized to the standard deviation in chemical shift strengths. Reduced Planck's constant $\hbar \equiv 1$.

## TODO

- [ ] Add rotation errors, phase errors
- [ ] Add ensemble measurement

In [1]:
import qutip as qt
import numpy as np

## Create system

In [98]:
delay = 1e-2  # time is relative to chemical shift strength
pulse_width = 5e-3
N = 3  # number of spins

In [122]:
def get_Hsys(dipolar_strength=1e-2):
    chemical_shifts = 2*np.pi * np.random.normal(scale=1, size=(N,))
    Hcs = sum(
        [qt.tensor(
            [qt.identity(2)]*i
            + [chemical_shifts[i] * qt.sigmaz()]
            + [qt.identity(2)]*(N-i-1)
        ) for i in range(N)]
    )
    # dipolar interactions
    dipolar_matrix = 2*np.pi * np.random.normal(scale=dipolar_strength, size=(N, N))
    Hdip = sum([
        dipolar_matrix[i, j] * (
            2 * qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmaz()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmaz()]
                + [qt.identity(2)]*(N-j-1)
            )
            - qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmax()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmax()]
                + [qt.identity(2)]*(N-j-1)
            )
            - qt.tensor(
                [qt.identity(2)]*i
                + [qt.sigmay()]
                + [qt.identity(2)]*(j-i-1)
                + [qt.sigmay()]
                + [qt.identity(2)]*(N-j-1)
            )
        )
        for i in range(N) for j in range(i+1, N)
    ])
    return Hcs + Hdip

In [120]:
X = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jx(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)
Y = sum(
    [qt.tensor(
        [qt.identity(2)]*i
        + [qt.spin_Jy(1/2)]
        + [qt.identity(2)]*(N-i-1)
    ) for i in range(N)]
)

In [124]:
def get_pulses(Hsys, X, Y, pulse_width, delay, rot_error=0):
    rot = np.random.normal(scale=rot_error)
    pulses = [
        qt.propagator(X * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(Y * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-X * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(-Y * (np.pi/2) * (1 + rot) / pulse_width + Hsys, pulse_width),
        qt.propagator(Hsys, pulse_width)
    ]
    delay_propagator = qt.propagator(Hsys, delay)
    pulses = [delay_propagator * i for i in pulses]
    return pulses

In [178]:
Hsys_ensemble = [get_Hsys() for _ in range(3)]

In [179]:
pulses_ensemble = [
    get_pulses(H, X, Y, pulse_width, delay, rot_error=0.01) for H in Hsys_ensemble
]

In [176]:
Utarget = qt.tensor([qt.identity(2)]*N)

In [143]:
pulse_names = [
    'X', 'Y', 'Xbar', 'Ybar', 'delay'
]

## Define computational tree

Keep track of propagator in memory to reduce computation time.

In [104]:
class Node(object):
    
    def __init__(self, propagator, sequence=[], depth=0):
        self.propagator = propagator
        self.sequence = sequence
        self.depth = depth
        
        self.children = {}
    
    def has_children(self):
        return len(self.children) > 0
    
    def evaluate(self, Utarget, pulses, reward_dict, max_depth=6):
        """If the node isn't at max_depth, then create children and
        evaluate each individually. If the node is at max_depth, then
        calculate the reward and add the sequence/reward pair to
        reward_dict.
        
        Arguments:
            pulses: An array of unitary operators representing all actions
                that can be applied to the system.
        Returns: The maximum reward seen by the node or its children, and
            the corresponding sequence.
            
        """
        if self.depth < max_depth:
            max_reward = 0
            max_reward_sequence = []
            for i, pulse in enumerate(pulses):
                propagator = pulse * self.propagator
                child = Node(propagator,
                             self.sequence + [i],
                             depth=self.depth + 1)
                r, s = child.evaluate(Utarget, pulses, reward_dict, max_depth)
                if r > max_reward:
                    max_reward = r
                    max_reward_sequence = s
            return max_reward, max_reward_sequence
        else:
            fidelity = np.clip(
                qt.metrics.average_gate_fidelity(self.propagator, Utarget),
                0, 1
            )
            reward = - np.log10(1.0 - fidelity + 1e-100)
            sequence_str = ','.join([str(a) for a in self.sequence])
            reward_dict[sequence_str] = reward
            return reward, self.sequence

In [171]:
class EnsembleNode(object):
    
    def __init__(self, propagators, sequence=[], depth=0):
        self.propagators = propagators  # will be list of Qobj for ensemble
        self.sequence = sequence
        self.depth = depth
        
        self.children = {}
    
    def has_children(self):
        return len(self.children) > 0
    
    def evaluate(self, Utarget, pulses_ensemble, reward_dict, max_depth=6):
        """If the node isn't at max_depth, then create children and
        evaluate each individually. If the node is at max_depth, then
        calculate the reward and add the sequence/reward pair to
        reward_dict.
        
        Arguments:
            pulses: An array of unitary operators representing all actions
                that can be applied to the system.
        Returns: The maximum reward seen by the node or its children, and
            the corresponding sequence.
            
        """
        if self.depth < max_depth:
            max_reward = 0
            max_reward_sequence = []
            for i in range(len(pulses_ensemble[0])):
                propagators = []
                for j in range(len(pulses_ensemble)):
                    propagators.append(pulses_ensemble[j][i] * self.propagators[j])
                child = EnsembleNode(propagators,
                             self.sequence + [i],
                             depth=self.depth + 1)
                r, s = child.evaluate(Utarget, pulses_ensemble, reward_dict, max_depth)
                if r > max_reward:
                    max_reward = r
                    max_reward_sequence = s
            return max_reward, max_reward_sequence
        else:
            fidelities = [np.clip(
                qt.metrics.average_gate_fidelity(p, Utarget),
                0, 1
            ) for p in self.propagators]
            fidelity = np.nanmean(fidelities)
            reward = - np.log10(1.0 - fidelity + 1e-100)
            sequence_str = ','.join([str(a) for a in self.sequence])
            reward_dict[sequence_str] = reward
            return reward, self.sequence

## Identify primitives

In [173]:
# reward_dict = {}
    
# root = Node(Utarget)
# max_reward, max_reward_sequence = root.evaluate(
#     Utarget, pulses, reward_dict
# )

# print(f'Max reward was {max_reward}\nSequence:\t{max_reward_sequence}')

In [180]:
reward_dict = {}
    
root = EnsembleNode([Utarget] * 3)
max_reward, max_reward_sequence = root.evaluate(
    Utarget, pulses_ensemble, reward_dict
)

print(f'Max reward was {max_reward}\nSequence:\t{max_reward_sequence}')

Max reward was 2.7779486738863004
Sequence:	[0, 3, 3, 0, 3, 3]


In [181]:
sequences = list(reward_dict.keys())

In [182]:
rewards = list(reward_dict.values())

In [183]:
sequences.sort(key=lambda s: -reward_dict[s])

In [184]:
sequences[:33]

['0,3,3,0,3,3',
 '1,0,0,1,0,0',
 '2,1,1,2,1,1',
 '3,2,2,3,2,2',
 '0,0,1,0,0,1',
 '1,1,2,1,1,2',
 '2,2,3,2,2,3',
 '3,3,0,3,3,0',
 '0,1,0,0,1,0',
 '1,2,1,1,2,1',
 '2,3,2,2,3,2',
 '3,0,3,3,0,3',
 '0,0,3,0,0,3',
 '1,1,0,1,1,0',
 '2,2,1,2,2,1',
 '3,3,2,3,3,2',
 '0,1,1,0,1,1',
 '0,3,0,0,3,0',
 '1,0,1,1,0,1',
 '1,2,2,1,2,2',
 '2,1,2,2,1,2',
 '2,3,3,2,3,3',
 '3,0,0,3,0,0',
 '3,2,3,3,2,3',
 '0,3,0,3,0,3',
 '1,0,1,0,1,0',
 '2,1,2,1,2,1',
 '3,2,3,2,3,2',
 '0,1,0,1,0,1',
 '1,2,1,2,1,2',
 '2,3,2,3,2,3',
 '3,0,3,0,3,0',
 '4,0,0,4,0,0']

In [187]:
[reward_dict[s] for s in sequences[:24]]

[2.7779486738863004,
 2.7779486738863004,
 2.7779486738863004,
 2.7779486738863004,
 2.7779486738862715,
 2.7779486738862715,
 2.7779486738862715,
 2.7779486738862715,
 2.777948673886214,
 2.777948673886214,
 2.777948673886214,
 2.777948673886214,
 2.777948673886185,
 2.777948673886185,
 2.777948673886185,
 2.777948673886185,
 2.777948673886098,
 2.777948673886098,
 2.777948673886098,
 2.777948673886098,
 2.777948673886098,
 2.777948673886098,
 2.777948673886098,
 2.777948673886098]

## Construct primitives

In [194]:
primitives_ensemble = []  # list of list of Qobj

for j in range(len(pulses_ensemble)):
    primitives = []
    for i in range(24):
        propagator = qt.identity(pulses_ensemble[0][0].dims[0])
        sequence = sequences[i]
        for p in sequence.split(','):
            propagator = pulses_ensemble[j][int(p)] * propagator
        primitives.append(propagator)
    primitives_ensemble.append(primitives)

In [215]:
# qt.hinton(primitives[0] + Utarget)
# qt.hinton(primitives[1] + Utarget)

## Build longer pulse sequences from primitives

In [200]:
reward_dict_2 = {}

root = EnsembleNode([Utarget]*3)
max_reward, max_reward_sequence = root.evaluate(
    Utarget, primitives_ensemble, reward_dict_2, max_depth=2)

In [201]:
sequences_2 = list(reward_dict_2.keys())
rewards_2 = list(reward_dict_2.values())

In [212]:
sequences_2.sort(key=lambda s: -reward_dict_2[s])

In [213]:
sequences_2[:17]

['0,16',
 '1,19',
 '2,21',
 '3,22',
 '4,14',
 '5,15',
 '6,12',
 '7,13',
 '16,0',
 '19,1',
 '21,2',
 '22,3',
 '12,6',
 '13,7',
 '14,4',
 '15,5',
 '4,3']

In [208]:
rewards_2.sort(reverse=True)

In [211]:
rewards_2[:17]

[4.138787196443834,
 4.138787196443834,
 4.138787196443834,
 4.138787196443834,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.1387871964425065,
 4.1387871964425065,
 4.1387871964425065,
 4.1387871964425065,
 4.034173108172478]

In [214]:
for i in range(16):
    inds = sequences_2[i].split(',')
    print(sequences[int(inds[0])] + ',' + sequences[int(inds[1])])

0,3,3,0,3,3,0,1,1,0,1,1
1,0,0,1,0,0,1,2,2,1,2,2
2,1,1,2,1,1,2,3,3,2,3,3
3,2,2,3,2,2,3,0,0,3,0,0
0,0,1,0,0,1,2,2,1,2,2,1
1,1,2,1,1,2,3,3,2,3,3,2
2,2,3,2,2,3,0,0,3,0,0,3
3,3,0,3,3,0,1,1,0,1,1,0
0,1,1,0,1,1,0,3,3,0,3,3
1,2,2,1,2,2,1,0,0,1,0,0
2,3,3,2,3,3,2,1,1,2,1,1
3,0,0,3,0,0,3,2,2,3,2,2
0,0,3,0,0,3,2,2,3,2,2,3
1,1,0,1,1,0,3,3,0,3,3,0
2,2,1,2,2,1,0,0,1,0,0,1
3,3,2,3,3,2,1,1,2,1,1,2
